**Importing necessary libraries and the dataset**

In [ ]:
import numpy as np
import pandas as pd

In [35]:
d1=pd.read_csv(r"C:\Users\adity\OneDrive\Desktop\bank_customers_dataset\bank_customers_churn_prediction.csv")
d1.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


**Data Cleaning**

The columns RowNumber, CustomerID, Surname are not required for predicting whether the customer stays or leaves

In [36]:
d1.drop('RowNumber',axis='columns',inplace=True)
d1.drop(['CustomerId','Surname'],axis='columns',inplace=True)
d1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [37]:
d1['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

As Geography has only 3 different values for rows, it can be replaced with 3 numbers ie 0 for france, 0.5 for Germany and 1 for Spain.
Gender column can also be replaced by 0 for Female and 1 for Male.

In [38]:
d1['Geography'].replace({'France': 0,'Germany': 0.5,'Spain': 1},inplace=True)
d1['Gender'].replace({'Female':0,'Male':1},inplace=True)
d1.head()

C:\Users\adity\AppData\Local\Temp\ipykernel_13208\2239524233.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  d1['Geography'].replace({'France': 0,'Germany': 0.5,'Spain': 1},inplace=True)
C:\Users\adity\AppData\Local\Temp\ipykernel_13208\2239524233.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0.0,0,42,2,0.00,1,1,1,101348.88,1
1,608,1.0,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0.0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0.0,0,39,1,0.00,2,0,0,93826.63,0
4,850,1.0,0,43,2,125510.82,1,1,1,79084.10,0


**Scaling**

For proper training of model, few rows have to be scaled between 0 and 1. The scaling is done using MinMaxScaling.

In [39]:
cols=['CreditScore','Age','Tenure','Balance','EstimatedSalary']
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
d1[cols]=scaler.fit_transform(d1[cols])
d1.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.538,0.0,0,0.324324,0.2,0.000000,1,1,1,0.506735,1
1,0.516,1.0,0,0.310811,0.1,0.334031,1,0,1,0.562709,0
2,0.304,0.0,0,0.324324,0.8,0.636357,3,1,0,0.569654,1
3,0.698,0.0,0,0.283784,0.1,0.000000,2,0,0,0.469120,0
4,1.000,1.0,0,0.337838,0.2,0.500246,1,1,1,0.395400,0


In [40]:
X=d1.drop('Exited',axis='columns')
y=d1['Exited']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=5)

In [41]:
len(X_train.columns)

10

Here,I am training the model to predict with X as all other columns and y as the required row. Here, I am building a neural network using Tensorflow with 3 layers ie 1 activation and 2 hidden layers.

In [42]:
import tensorflow as tf
from tensorflow import keras

mod=keras.Sequential([
    keras.layers.Dense(10,input_shape=(10,),activation='relu'),
    keras.layers.Dense(6,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])
mod.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])
mod.fit(X_train,y_train,epochs=100)

Epoch 1/100


c:\Users\adity\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7875 - loss: 0.5362
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7945 - loss: 0.4947
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8001 - loss: 0.4746
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7964 - loss: 0.4635
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7992 - loss: 0.4483
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8143 - loss: 0.4169
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8027 - loss: 0.4249
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8081 - loss: 0.4095
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8102 - loss: 0.4020
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8101 - loss: 0.3970
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8296 - loss: 0.3839
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

In [43]:
yp=mod.predict(X_test)
y_pred=[]
for element in yp:
    if element>0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


Here, I am checking the model accuracy and other metrics using actual data and data predicted by model.

In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.97      0.91      1595
           1       0.76      0.38      0.51       405

    accuracy                           0.85      2000
   macro avg       0.81      0.68      0.71      2000
weighted avg       0.84      0.85      0.83      2000



Here, my model has shown accuracy of 85% and average f1 score of 0.71. 